# Data Preprocessing
### __과정__
**csv dataset 불러오기 -> Preprocessing -> Pretraining Dataset 구축(합치기) -> Additional Preprocessing**<br>
* __Preprocessing__
    1. tag 제거 (3가지 유형)
    2. 한개의 문단으로 이루어진 data를 문장으로 구분된 data로 만들기
    3. 다중 띄어쓰기 제거
* __Additional Preprocessing__
    1. Null 값 제거
    2. 영어, 숫자, 연산 기호만 남기기
    3. OOM 방지를 위한 데이터 분할

In [1]:
# DATA version
VERSION = ""

# PATH
# INPUT PATH
PAPER_PATH = '../Dataset/Datacon/paper.csv'
KoSAT_PATH = '../Dataset/WebCrawling/수능/ICES_sooneung.csv'
NEWS_PATH = '../Dataset/WebCrawling/news/ICES_News.csv'
VOCA_DIC_PATH = ""

# OTUPUT PATH
FINAL_DATA_CSV_PATH = '../Dataset/FinalDataset_v{}.csv'.format(VERSION)
FINAL_DATA_TXT_PATH = "rsc/training_data/final_file/AllData_full_text_v{}.txt".format(VERSION)
FINAL_DATA_TXT_SETENCE_PATH = "rsc/training_data/final_file/AllData_sentence_v{}.txt".format(VERSION)
DISTRIBUTED_FINAL_DATA_TXT_SETENCE_PATH = 'rsc/training_data/final_file/AllData_sentence_v{}_{}.txt'

# version
VERSION = "4"

In [2]:
import csv
csv.field_size_limit(100000000)

import pandas as pd
import random

In [3]:
## 뉴스 파일 -> xlsx파일이므로 csv파일로 다시 저장하여 load함
# data_xls = pd.read_excel('../Dataset/WebCrawling/news/ICES_news.xlsx')
# data_xls.to_csv('../Dataset/WebCrawling/news/ICES_News.csv', encoding='utf-8', index=False)
## 단어사전 파일 -> xlsx파일이므로 csv파일로 다시 저장하여 load함
# for i in file_path:
#     data_xls = pd.read_excel('../Dataset/WebCrawling/dictionary/'+i)
#     data_xls.to_csv('../Dataset/WebCrawling/dictionary/'+i[:-4]+'csv', encoding='utf-8', index=False)

## 1. csv dataset 불러오기
**논문, 수능, 뉴스, 단어사전 csv files**

In [4]:
# 논문 dataset
paper_data = pd.read_csv(PAPER_PATH)
paper_data.drop(['Unnamed: 0'], axis='columns', inplace=True)
# 수능 dataset
koSAT = pd.read_csv(KoSAT_PATH)
# 웹 크롤링 - 뉴스 dataset
news = pd.read_csv(NEWS_PATH)
# 웹 크롤링 - 단어사전 dataset
import os
file_path = [i for i in os.listdir('../Dataset/WebCrawling/dictionary') if i != '.ipynb_checkpoints' and 'xlsx' not in i]
vocab_dic = pd.DataFrame()
for i in file_path:
    df4 = pd.read_csv('../Dataset/WebCrawling/dictionary/'+i[:-4]+'.csv')
    vocab_dic =  pd.concat([vocab_dic, df4])

In [5]:
nPaper = len(paper_data) # 45000
nKoSATlen = len(koSAT) # 16
nNews = len(news) # 1356
nVocabDic = len(vocab_dic) # 54827
total = nPaper + nKoSATlen + nNews + nVocabDic
print(total)

101199


## 2. Preprocessing
**전처리 방법**
1. tag 제거
2. 영어, 숫자, 연산 기호만 남기기
3. 다중 띄어쓰기 제거

In [6]:
import re

def pre_processing(csv_file, key_name):
    csv_file[key_name] = csv_file[key_name].astype(str)
    # HTML tag 제거
    # tag 제거1 => <p>
    csv_file[key_name] = csv_file[key_name].map(lambda x: re.sub('(<([^>]+)>)', ' ',x.replace('\n',' ')))
    # tag 제거2 # => $nbps;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('&nbsp;',' '))
    # tag 제거3 # => $ensp;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('&ensp;',' '))
    # tag 제거4 # => $emsp;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('&emsp;',' '))
    # tag 제거5 # => $amp;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('$amp;','&'))
    # tag 제거6 # => $lt;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('$lt;','<'))
    # tag 제거7 # => $gt;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('$gt;','>'))
    # tag 제거8 # => $quot;
    csv_file[key_name] = csv_file[key_name].map(lambda x: x.replace('$quot;','\"'))
    # tag 제거9 # => $rsps;
    csv_file[key_name] = csv_file[key_name].map(lambda x: re.sub(r'&(/?)(.+?);','',x))
    # tag 제거10 => #$1{\\beta}$
    csv_file[key_name] = csv_file[key_name].map(lambda x: re.sub(r'\$[^)]*\$', '', x))
    
    # 한자 제거 
    csv_file[key_name] = csv_file[key_name].map(lambda x: re.sub(r'[一-龥]','',x))
    # 영어, 숫자, 연산 기호만 남기기
    csv_file[key_name] = csv_file[key_name].map(lambda x: re.sub('[^a-zA-Z0-9ㄱ-ㅣ가-힣+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x().,%#{}]',' ',x))
#     다중 띄어쓰기 제거
    csv_file[key_name] = csv_file[key_name].map(lambda x: re.sub(' +',' ', x))
    
    return csv_file

In [7]:
# 각 데이터 전처리 진행
paper_data = pre_processing(paper_data, 'PAPER_FULL_TXT')
koSAT = pre_processing(koSAT, 'text')
news = pre_processing(news, 'content')
vocab_dic = pre_processing(vocab_dic, 'content')

In [8]:
## 전처리된 CSV FILE 저장이 필요할 경우 진행 
## 논문데이터만 활용하기 위한 csv
paper_data.loc[:,['PAPER_FULL_TXT']].to_csv('only_paper_data/preprocessed_paper_v{}.csv'.format(VERSION),mode='w',index=False)

## 3. Pretraining Dataset 구축
**Dataset column 이름 통일**

In [9]:
# dataset 합치기 위해 columns 이름 통일('PAPER_FULL_TXT')
koSAT = koSAT.rename(columns={'text': 'PAPER_FULL_TXT'})
news = news.rename(columns={'content': 'PAPER_FULL_TXT'})
vocab_dic = vocab_dic.rename(columns={'content': 'PAPER_FULL_TXT'})
# 합친후 인덱스 재설정 및 나쁜 데이터 제거(이후 전처리 과정중 발견)
vocab_dic = vocab_dic.reset_index(drop=True)
vocab_dic = vocab_dic.drop(vocab_dic.index[[1705, 9245, 10743, 11060, 11800, 20137, 42020]])

**csv File 생성**

In [10]:
# dataset 합치기
paper_data.loc[:,['PAPER_FULL_TXT']].to_csv(FINAL_DATA_CSV_PATH,mode='w',index=False)
koSAT.loc[:,['PAPER_FULL_TXT']].to_csv(FINAL_DATA_CSV_PATH,mode='a',index=False)
news.loc[:,['PAPER_FULL_TXT']].to_csv(FINAL_DATA_CSV_PATH,mode='a',index=False)
vocab_dic.loc[:,['PAPER_FULL_TXT']].to_csv(FINAL_DATA_CSV_PATH,mode='a',index=False)

In [11]:
final = pd.read_csv(FINAL_DATA_CSV_PATH)
final

,PAPER_FULL_TXT
0,구획화재에서 환기조건의 변화가 화재특성에 미치는 영향 (Part II 다차원 화재...
1,방화셔터를 이용한 아트리움 제연과 피난안전에 관한 수치해석 연구 A Numeric...
2,이중격실 Pool 화재에 대한 FDS 검증분석 Validation of FDS f...
3,실규모 구획화재의 연소가스에 대한 혼합분율 분석 Mixture Fraction A...
4,C 60 (CH 2 ) n OH (n=0 2)와 C 60 (OH) 2 의 분자구조...
...,...
101190,"사용분야 기타(Optical Holography, Infrared Examinati..."
101191,수상 노벨 물리학상(1952년) 목차노벨상 수상자 필릭스 블로흐(Felix Bloc...
101192,학명 Pseudohiatula esculenta (Wulf.ex Fr.)Sing. ...
101193,두 장 이상의 천에 같은 모양의 바느질 표시를 할 때 사용하는 방법. 바느질할 선을...


In [12]:
## 논문 데이터만 이용하기위함
final = pd.read_csv('only_paper_data/preprocessed_paper_v{}.csv'.format(VERSION))
final

,PAPER_FULL_TXT
0,구획화재에서 환기조건의 변화가 화재특성에 미치는 영향 (Part II 다차원 화재...
1,방화셔터를 이용한 아트리움 제연과 피난안전에 관한 수치해석 연구 A Numeric...
2,이중격실 Pool 화재에 대한 FDS 검증분석 Validation of FDS f...
3,실규모 구획화재의 연소가스에 대한 혼합분율 분석 Mixture Fraction A...
4,C 60 (CH 2 ) n OH (n=0 2)와 C 60 (OH) 2 의 분자구조...
...,...
44995,공공기관 행정정보시스템 관련 단위과제 및 보존기간 책정 현황분석 Current S...
44996,드론기반 도심지 디지털트윈 3차원 모형 구축에 관한 연구 A Study on 3D...
44997,공공기관 기록관리 평가제도 개선 방안 자체평가제도 도입을 중심으로 Improvem...
44998,북한의 일상생활과 문화 아카이빙 Cultural Archiving of Every...


## 4. Aditional Preprocessing
**Null 값 제거**

In [13]:
# NaN 값 제거, index 재설정 후 저장
final = final[~final['PAPER_FULL_TXT'].isnull()]
final = final.reset_index(drop=True)
# final.loc[:,['PAPER_FULL_TXT']].to_csv(FINAL_DATA_CSV_PATH,mode='w',index=False)

# data shuffle
data = final['PAPER_FULL_TXT'].to_list()
random.shuffle(data)
print(len(data))
print(final)

45000
                                          PAPER_FULL_TXT
0       구획화재에서 환기조건의 변화가 화재특성에 미치는 영향 (Part II 다차원 화재...
1       방화셔터를 이용한 아트리움 제연과 피난안전에 관한 수치해석 연구 A Numeric...
2       이중격실 Pool 화재에 대한 FDS 검증분석 Validation of FDS f...
3       실규모 구획화재의 연소가스에 대한 혼합분율 분석 Mixture Fraction A...
4       C 60 (CH 2 ) n OH (n=0 2)와 C 60 (OH) 2 의 분자구조...
...                                                  ...
44995   공공기관 행정정보시스템 관련 단위과제 및 보존기간 책정 현황분석 Current S...
44996   드론기반 도심지 디지털트윈 3차원 모형 구축에 관한 연구 A Study on 3D...
44997   공공기관 기록관리 평가제도 개선 방안 자체평가제도 도입을 중심으로 Improvem...
44998   북한의 일상생활과 문화 아카이빙 Cultural Archiving of Every...
44999   근전계 RCS 측정 오차 요인 및 불확도 분석 An Analysis of Erro...

[45000 rows x 1 columns]


**txt file 생성**<br>
pretraining에 필요한 vocab 사전 만들기 위한 text file

In [14]:
# 논문데이터만 활용하기 위한 변수 변경
FINAL_DATA_TXT_PATH = "only_paper_data/paper_full_text_v{}.txt".format(VERSION)
FINAL_DATA_TXT_SETENCE_PATH = "only_paper_data/paper_full_sentence_v{}.txt".format(VERSION)
DISTRIBUTED_FINAL_DATA_TXT_SETENCE_PATH = 'only_paper_data/paper_sentence_v{}_{}.txt'

In [15]:
# index별로 \n로 구분
f = open(FINAL_DATA_TXT_PATH, 'w')
# PC 성능이 좋을 경우
for i in range(len(final)):
    f.write(final.loc[i].values[0].strip()+'\n')
    f.write('\n')
# # PC 성능이 좋지않아 나누어서
# data_cut = 27
# max_len = len(final)//data_cut
# for i in range(data_cut):
#     for j in range(max_len):
#         f.write(data.loc[(i*max_len) + j].values[0] + '\n')

f.close()
print("Create complete : {}".format(FINAL_DATA_TXT_PATH))

Create complete : only_paper_data/paper_full_text_v4.txt


In [16]:
## Test page입니다
f = open(FINAL_DATA_TXT_PATH, 'r')
d = f.readlines()
count = 0
# for i in d:
#     if i == '\n': count += 1
# print(count)
# print(len(d))
# print(d.count('\n'))

for i in d:
    print(i)
    if count == 3: break
    count+=1
f.close()

구획화재에서 환기조건의 변화가 화재특성에 미치는 영향 (Part II 다차원 화재거동) Effects of Ventilation Condition on the Fire Characteristics in Compartment Fires (Part II Multi-dimensional Fire Dynamics) 김종현 고권현 박충화 황철홍 초록 실규모 ISO 9705 표준 화재실에서 환기조건 변화에 따른 다차원 화재거동에 관한 수치해석적 연구가 수행되었다. 선행된 실험과 동일한 조건에 대하여 FDS(Fire Dynamic Simulator)가 사용되었다. 과환기화재 및 환기부족화재의 발생을 위하여 연료 유량과 출입구의 폭이 변화되었다. 주요 결과로서, 환기부족화재의 내부 유동패턴은 과환기화재와 비교할 때 반대방향을 갖으며, 그 결과 다량의 고온 생성물이 구획내부에서 재순환되는 매우 중요한 특징을 확인하였다. 환기조건에 따른 유동패턴의 변화는 구획 내부에서 고온 생성물의 체류시간을 크게 변화시키며, CO 및 그을음의 복잡한 생성과정에 큰 영향을 미칠 수 있다. 환기부족화재는 구획 내부의 열 및 유동구조 뿐만 아니라 화학종의 분포에 관하여 매우 복잡한 3차원 구조를 생성하였다. 특히, 구획 내부의 측면에서 추가적인 반응은 유동패턴 및 CO 생성에 매우 큰 영향을 주고 있다. 복잡한 CO의 분포는 3차원 산소 농도의 분포 및 유동 패턴을 통해 체계적으로 분석되었다. 위 결과로 부터 고온 상층부에서 측정된 국부 화학종 농도는 구획 내부의 화재특성을 규명하는데 많은 한계가 있음을 확인할 수 있었다. Multi-dimensional fire dynamics were studied numerically with the change in ventilation conditions in a full-scale ISO 9705 room. Fire Dynamic Simulator (FDS) was used for the identical conditions conducted in pr

**문단을 문장단위로 자르기(\n)**<br>
text파일은 *1줄에 data 1개*가 들어있다.<br>
*1줄에 1문장*으로 변경 (data사이에는 \n으로 구분)

In [114]:
a="asdfk wlfwlk nflwng %#$ kkklw%#$sfk %#$dlkfjlsj lkfjsl%#$ "
re.findall("%#\$", a)
# asfkljsf%#$faskld``fjlka%#$  %#\$
b="faklsdfjlk %#$"
re.sub("%#\$", "세", b)
b.find('%#$')


11

In [17]:
f=open(FINAL_DATA_TXT_PATH, 'r')
lines=f.readlines()
f.close()
count = 0
for i, d in enumerate(lines):
    count += len(re.findall("%#\$",d))
    
print(count)

0


In [18]:
# URL의 경우 예외처리 @@로 한 후에 나중에 다시 변경
import re
URL_reguler_expression = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
URL_REMOVER = re.compile(URL_reguler_expression)

f=open(FINAL_DATA_TXT_PATH, 'r')
# g=open("rsc/training_data/final_text_url_removed.txt", 'w')
# # 논문 데이터만 사용하기위함
g=open("only_paper_data/paper_full_url_removed_v{}.txt".format(VERSION), 'w')

lines=f.readlines()
URL_li = []
new_line = ""

for line in lines:
    URL_li.extend(URL_REMOVER.findall(line))
    new_line = URL_REMOVER.sub("%#$", line)
    g.write(new_line)
    
f.close()
g.close()
print("find url : {}개".format(len(URL_li)))

find url : 9117개


In [19]:
# 문단을 문장단위로 자르기(\n)
def paragraph_to_sentence(path_paragraph, path_sentence):
    f = open(path_paragraph, 'r')
    g = open(path_sentence, 'w')
    
    line_num = 1
    
    level1 = 0
    level2 = 0
    level3 = 0
    level4 = 0
    level5 = 0

#     text = f.read()   ## 파일이 클경우 index가 초과해서 실행 안됨
    text = f.readline()
    while text:
        if line_num % 1000 == 0:
            print("processing line : {}".format(line_num))
        
        try:
            while(1):
                temp = text.find(".")
                # .을 못찾을 경우
                if temp == -1:
                    g.write(text)
                    break
                # .으로 끝날 경우((문자).(공백))
                elif (text[temp-1].isalpha() and text[temp+1] == " "):
                    level1 += 1
                    if(text[temp-2:temp+1] == "No." or text[temp-2:temp+1] == "no."): 
                        g.write(text[:temp+2])
                        text = text[temp+2:]
                    elif(text[temp-3:temp+1] == "Vol." or text[temp-3:temp+1] == "vol."): 
                        g.write(text[:temp+2])
                        text = text[temp+2:]
                    elif(text[temp-3:temp+1] == "Ver." or text[temp-3:temp+1] == "ver."): 
                        g.write(text[:temp+2])
                        text = text[temp+2:]
                    elif(text[temp-3:temp+1] == "Fig." or text[temp-3:temp+1] == "fig."): 
                        g.write(text[:temp+2])
                        text = text[temp+2:]
                    elif(text[temp-3:temp+1] == "Avg." or text[temp-3:temp+1] == "avg."): 
                        g.write(text[:temp+2])
                        text = text[temp+2:]
                    elif((text[temp-5:temp+1] == "Table." or text[temp-5:temp+1] == "table.") and text[temp+2].isdigit() and text[temp+3] == ' '): 
                        g.write(text[:temp+2])
                        text = text[temp+2:]
                    else:    
                        g.write(text[:temp+1] + "\n")
                        text = text[temp+2:]
                # (문자).(공백)안지키고 (문자).(문자)의 경우 줄 바꿈
                elif (text[temp-1].isalpha() and text[temp+1].isalpha()):
                    g.write(text[:temp+1] + "\n")
                    level2 += 1
                    text = text[temp+1:]
                elif (text[temp-1].isalpha() and text[temp+1].isdigit() and text[temp+2] == " "):
                    g.write(text[:temp+1] + "\n")
                    level3 += 1
                    text = text[temp+1:]
                else:
                    g.write(text[:temp+1])
                    level4 += 1
                    text = text[temp+1:]
        except IndexError:
            level5 += 1
            if text[-1] == '\n': g.write(text[:])
            else : 
                g.write(text[:] + "\n")
        
        text = f.readline()
        line_num += 1
    
    f.close()
    g.close()
    print("level 1 : {}, level 2 : {}, level 3 : {}, level 4 : {}, level 5 : {}".format(level1, level2, level3, level4, level5))
    print("Total line number(number of dataset) : {}".format(line_num - 1))
    print("create : ",path_sentence)

In [20]:
full_text_file = "only_paper_data/paper_full_url_removed_v{}.txt".format(VERSION)
sentence_text_file = "only_paper_data/paper_full_sentence_temp_v{}.txt".format(VERSION)
# # 논문 데이터만 사용하기 위함
# full_text_file = "rsc/training_data/final_text_url_removed.txt"
# sentence_text_file = "rsc/training_data/paper_full_sentence_temp.txt"
paragraph_to_sentence(full_text_file, sentence_text_file)

processing line : 1000
processing line : 2000
processing line : 3000
processing line : 4000
processing line : 5000
processing line : 6000
processing line : 7000
processing line : 8000
processing line : 9000
processing line : 10000
processing line : 11000
processing line : 12000
processing line : 13000
processing line : 14000
processing line : 15000
processing line : 16000
processing line : 17000
processing line : 18000
processing line : 19000
processing line : 20000
processing line : 21000
processing line : 22000
processing line : 23000
processing line : 24000
processing line : 25000
processing line : 26000
processing line : 27000
processing line : 28000
processing line : 29000
processing line : 30000
processing line : 31000
processing line : 32000
processing line : 33000
processing line : 34000
processing line : 35000
processing line : 36000
processing line : 37000
processing line : 38000
processing line : 39000
processing line : 40000
processing line : 41000
processing line : 42000
p

In [21]:
import copy
URL = copy.deepcopy(URL_li)

In [22]:
f = open(sentence_text_file, 'r')
d = f.readlines()
f.close()
count = 0
for i, l in enumerate(d):
    count += len(re.findall("%#\$", l))
        
print("예상 {}개, 결과 : {}".format(len(URL_li), count))

예상 9117개, 결과 : 9117


In [23]:
f = open(sentence_text_file, 'r')
d = f.readlines()
f.close()
count = 0
for i ,l in enumerate(d):
    if l == '\n': count += 1
print(count)

45000


In [24]:
URL.reverse()
temp_str = ""

g = open(FINAL_DATA_TXT_SETENCE_PATH, 'w')
try:
    for i, line in enumerate(d):
        while(1):
#                 if ("``" in line) : print(line)
            start = line.find('%#$')
            if start == -1: 
                g.write(line)
                break
            g.write(re.sub("%#\$", URL.pop(), line[:start+3]))
            line = line[start+3:]
                
    print("complete")
except IndexError:
    print("IndexError : line {}".format(i))
    
f.close()
g.close()
# 총 6169673줄의 text 파일 -> URL 처리후 6106379줄

complete


In [25]:
len(URL)

0

In [26]:
a = open(FINAL_DATA_TXT_SETENCE_PATH, 'r')
b = a.readlines()
## 공백 개수 확인 (예상 101189 - data 갯수)
count = 0
for i in b:
    if i == '\n': 
        count += 1
a.close()
print("예상 : 101189, 결과 : {}".format(count))

예상 : 101189, 결과 : 45000


In [27]:
u = open(FINAL_DATA_TXT_SETENCE_PATH, 'r')
z = u.readlines()
u.close()
print("Total line number of text file : ", len(z))

Total line number of text file :  5233444


**text file 분할**<br>
원본 text file은 크기가 너무 커서 OOM 이슈 발생

In [28]:
# 총 dataset 개수와 일치하는지 확인 : 101189개
f = open(FINAL_DATA_TXT_SETENCE_PATH, 'r')
d = f.readlines()
f.close()
len([i for i in d if i == '\n'])

45000

In [29]:
# 공백으로 나누어진 1개의 데이터별로 분리
result = []
temp = []
for i in d:
    if i == '\n':
        result.append(temp)
        result.append('\n')
        temp = []
    else:
        temp.append(i)
        
len(result)  # 데이터 개수 확인 (101189개)

90000

In [79]:
# Finalresult_n.txt 파일 생성
for i in range(0, len(result), 100000):
    print(i)
    f = open(DISTRIBUTED_FINAL_DATA_TXT_SETENCE_PATH.format(VERSION, ((i+1)/100000)+1), 'w')
    if i == 100000: num = len(result) - 100000
    else: num = 10000
    for j in range(num):
        for k in result[i+j]:
            f.write(k)
    f.close()

SyntaxError: invalid syntax (<ipython-input-79-756fc3462bfa>, line 5)

In [30]:
# 논문데이터 분할 파일 생성
cutting_line = 9000

for i in range(0, len(result), cutting_line):
    print("step : ", i)
    f = open(DISTRIBUTED_FINAL_DATA_TXT_SETENCE_PATH.format(VERSION, ((i+1) // cutting_line) + 1), 'w')
    if (i + cutting_line) > len(result): num = len(result) - i -1
    else: num = cutting_line
    for j in range(num):
        for k in result[i+j]:
            f.write(k)
    f.close()

step :  0
step :  9000
step :  18000
step :  27000
step :  36000
step :  45000
step :  54000
step :  63000
step :  72000
step :  81000
